In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

#load initial csv
initial_data = pd.read_csv('Taxi_Trips.csv')


#select desired rows
cab_fare_df = initial_data.iloc[: ,[2,3,4,5,10,13,16,17,18,20,21]]
#print(cab_fare_df.columns)

#print(cab_fare_df.dtypes)
#print(cab_fare_df.describe)

#drop rows with NA values
cab_fare_df = cab_fare_df.dropna()

#drop duplicates
cab_fare_df.drop_duplicates(inplace = True)

#print(cab_fare_df.head(5))


777204


In [30]:
#filtering out problematic rows

#filter latitudes outside of continental US
cab_fare_df = cab_fare_df[(cab_fare_df.iloc[:, 7].between(25.0, 49.0)) & (cab_fare_df.iloc[:, 9].between(25.0, 49.0))]

#filter longitudes outside of continental US
cab_fare_df = cab_fare_df[(cab_fare_df.iloc[:, 8].between(-125, -67)) & (cab_fare_df.iloc[:, 10].between(-125, -67))]

#filter miles
cab_fare_df = cab_fare_df[cab_fare_df['Trip Seconds'].between(120.0, 21600.0)]

#filter seconds
cab_fare_df = cab_fare_df[cab_fare_df['Trip Miles'].between(1.0, 1000.0)]
#print(cab_fare_df.head(5))

In [31]:
#sum fare and extras columns 
cab_fare_df['Trip Cost'] = cab_fare_df['Fare'] + cab_fare_df['Extras']

In [33]:
#convert timestamp columns

cab_fare_df['Trip Start Timestamp'] = pd.to_datetime(cab_fare_df['Trip Start Timestamp'], format="%m/%d/%Y %I:%M:%S %p")
#cab_fare_df['Trip End Timestamp'] = pd.to_datetime(cab_fare_df['Trip End Timestamp'])

#create month column 
cab_fare_df['Month'] = cab_fare_df['Trip Start Timestamp'].dt.month

#create hour column
cab_fare_df['Hour'] = cab_fare_df['Trip Start Timestamp'].dt.hour 

print(cab_fare_df.head(5))
print(cab_fare_df.count)

  Trip Start Timestamp      Trip End Timestamp  Trip Seconds  Trip Miles  \
1           2024-03-01  03/01/2024 12:15:00 AM         900.0        3.00   
3           2024-03-01  03/01/2024 12:30:00 AM        1770.0       13.36   
4           2024-03-01  03/01/2024 12:15:00 AM         849.0        6.13   
6           2024-03-01  03/01/2024 12:15:00 AM        1416.0       11.17   
7           2024-03-01  03/01/2024 12:15:00 AM        1553.0       13.21   

    Fare  Extras                    Company  Pickup Centroid Latitude  \
1  12.00     0.0  Taxi Affiliation Services                 41.901207   
3  34.75     4.0                   Sun Taxi                 41.980264   
4  18.51     0.0                5 Star Taxi                 41.899602   
6  29.75     0.0                5 Star Taxi                 41.874005   
7  34.00     4.0               City Service                 41.980264   

   Pickup Centroid Longitude  Dropoff Centroid Latitude  \
1                 -87.676356                 